In [2]:
import pandas as pd
import numpy as np
import math
import yaml
from pathlib import Path

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)

from db_psql_model import DatabaseCursor

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"

MoM FFBL Weekly Rankings Replacement 

In [ ]:
"""
MoM FFBL Weekly Rankings Replacement 2pt system
"""
matchups_query = """SELECT * FROM dev.reg_season_matchups"""
weekly_points_query = """SELECT * FROM dev.weekly_team_pts"""
teams_query = """SELECT team_key, name, nickname, game_id FROM dev.league_teams"""
settings_query = """SELECT * FROM dev.league_settings"""
matchups = (
    DatabaseCursor(PATH, options=OPTION_DEV)
    .copy_data_from_postgres(matchups_query)
    .drop_duplicates()
)

weekly_points = (
    DatabaseCursor(PATH, options=OPTION_DEV)
    .copy_data_from_postgres(weekly_points_query)
    .drop_duplicates()
)

weekly_points["team_key"] = (
    weekly_points["game_id"].astype(str)
    + ".l."
    + weekly_points["league_id"].astype(str)
    + ".t."
    + weekly_points["team_id"].astype(str)
)
weekly_points.drop_duplicates(subset=["team_key", "week"], inplace=True)

teams = (
    DatabaseCursor(PATH, options=OPTION_DEV)
    .copy_data_from_postgres(teams_query)
    .drop_duplicates()
)

settings = (
    DatabaseCursor(PATH, options=OPTION_DEV)
    .copy_data_from_postgres(settings_query)
    .drop_duplicates()
)

matchups_a = matchups.copy()
matchups_b = matchups.copy()

matchups_b_cols = list(matchups_b.columns)

rename_columns = {}
for col in matchups_b_cols:
    if "team_a" in col:
        rename_columns[col] = f"team_b{col[6:]}"
    elif "team_b" in col:
        rename_columns[col] = f"team_a{col[6:]}"

matchups_b.rename(columns=rename_columns, inplace=True)

matchups = pd.concat([matchups_a, matchups_b])

matchups.sort_values(["week_start", "team_a_team_key"], inplace=True)

matchups.reset_index(drop=True, inplace=True)

matchups.drop(
    [
        "is_matchup_recap_available",
        "is_tied",
        "matchup_recap_title",
        "matchup_recap_url",
        "status",
        "league_id",
        "team_a_grade",
        "team_b_grade",
    ],
    axis=1,
    inplace=True,
)

game_keys = list(matchups["game_id"].unique())

for key in game_keys:

    playoff_start_week = settings["playoff_start_week"][
        settings["game_id"] == key
    ].values[0]

    max_teams = settings["max_teams"][settings["game_id"] == key].values[0]

    one_reg_season = matchups[
        (matchups["game_id"] == key) & (matchups["week"] < playoff_start_week)
    ]

    one_reg_season["win_loss"] = np.where(
        one_reg_season["winner_team_key"] == one_reg_season["team_a_team_key"], "W", "L"
    )

    one_reg_season["pts_system_weekly_rank"] = one_reg_season.groupby(
        ["week", "game_id"]
    )["team_a_points"].rank("first", ascending=False)

    one_reg_season["pts_score"] = np.where(
        one_reg_season["pts_system_weekly_rank"] <= 5, 1, 0
    )

    one_reg_season = one_reg_season.merge(
        teams, how="left", left_on="team_a_team_key", right_on="team_key"
    )

    one_reg_season["team_a_name"] = one_reg_season["name"].fillna(
        one_reg_season["team_a_team_key"]
    )
    one_reg_season["team_a_nickname"] = one_reg_season["nickname"].fillna(
        one_reg_season["team_a_team_key"]
    )

    one_reg_season["game_id_a"] = one_reg_season["game_id_x"].fillna(
        one_reg_season["game_id_y"]
    )

    one_reg_season.drop(["name", "nickname"], axis=1, inplace=True)

    one_reg_season = one_reg_season.merge(
        teams, how="left", left_on="team_b_team_key", right_on="team_key"
    )

    one_reg_season["team_b_name"] = one_reg_season["name"].fillna(
        one_reg_season["team_b_team_key"]
    )
    one_reg_season["team_b_nickname"] = one_reg_season["nickname"].fillna(
        one_reg_season["team_b_team_key"]
    )

    one_reg_season.drop(["name", "nickname"], axis=1, inplace=True)

    one_reg_season["game_id"] = one_reg_season["game_id_a"]

    one_reg_season = one_reg_season.rename(
        columns={
            "team_a_team_key": "team_key",
            "team_a_name": "team_name",
            "team_a_nickname": "team_nickname",
            "team_a_points": "team_pts",
            "team_a_projected_points": "team_pro_pts",
            "team_b_team_key": "opp_team_key",
            "team_b_name": "opp_team_name",
            "team_b_nickname": "opp_team_nickname",
            "team_b_points": "opp_pts",
            "team_b_projected_points": "opp_pro_pts",
        }
    )

    one_reg_season = one_reg_season[
        [
            "game_id",
            "team_key",
            "team_name",
            "team_nickname",
            "week",
            "win_loss",
            "pts_score",
            "team_pts",
            "team_pro_pts",
            "opp_team_key",
            "opp_team_name",
            "opp_team_nickname",
            "opp_pts",
            "opp_pro_pts",
        ]
    ]

    one_reg_season["ttl_pts_for_run"] = one_reg_season.groupby(["team_key"])[
        "team_pts"
    ].cumsum()
    one_reg_season["pts_for_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pts_for_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    one_reg_season["ttl_pro_pts_for_run"] = one_reg_season.groupby(["team_key"])[
        "team_pro_pts"
    ].cumsum()
    one_reg_season["pro_pts_for_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pro_pts_for_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    one_reg_season["ttl_pts_agst_run"] = one_reg_season.groupby(["team_key"])[
        "opp_pts"
    ].cumsum()
    one_reg_season["pts_agst_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pts_agst_run"]
        .rank(method="max", ascending=True)
        .astype(int)
    )

    one_reg_season["ttl_pro_pts_agst_run"] = one_reg_season.groupby(["team_key"])[
        "opp_pro_pts"
    ].cumsum()
    one_reg_season["pro_pts_agst_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_pro_pts_agst_run"]
        .rank(method="max", ascending=True)
        .astype(int)
    )

    one_reg_season["ttl_wins_run"] = (
        one_reg_season["win_loss"].eq("W").groupby(one_reg_season["team_key"]).cumsum()
    )
    one_reg_season["ttl_loss_run"] = (
        one_reg_season["win_loss"].eq("L").groupby(one_reg_season["team_key"]).cumsum()
    )
    one_reg_season["w_l_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_wins_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    one_reg_season["ttl_pts_score_run"] = one_reg_season.groupby(["team_key"])[
        "pts_score"
    ].cumsum()
    one_reg_season["ttl_2pt_score_run"] = (
        one_reg_season["ttl_pts_score_run"] + one_reg_season["ttl_wins_run"]
    )
    one_reg_season["2pt_score_rank_run"] = (
        one_reg_season.groupby(["week"])["ttl_2pt_score_run"]
        .rank(method="min", ascending=False)
        .astype(int)
    )

    if key >= 390:
        one_reg_season["tuple"] = one_reg_season[
            ["2pt_score_rank_run", "pts_for_rank_run"]
        ].apply(tuple, axis=1)
    else:
        one_reg_season["tuple"] = one_reg_season[
            ["w_l_rank_run", "pts_for_rank_run"]
        ].apply(tuple, axis=1)

    one_reg_season.insert(
        5,
        "reg_season_rank_run",
        one_reg_season.groupby(["week"])["tuple"]
        .rank(method="min", ascending=True)
        .astype(int),
    )

    one_reg_season.sort_values(
        ["week", "reg_season_rank_run"], ascending=[True, True], inplace=True
    )

    reg_season_rank = one_reg_season[
        ["game_id", "team_key", "team_name", "team_nickname", "reg_season_rank_run"]
    ].tail(max_teams)

    one_playoff_season = weekly_points[
        (weekly_points["game_id"] == key)
        & (weekly_points["week"] >= playoff_start_week)
    ]
    one_playoff_season = one_playoff_season.merge(
        teams, how="left", left_on="team_key", right_on="team_key"
    )
    one_playoff_season = one_playoff_season[
        [
            "game_id_x",
            "team_key",
            "name",
            "nickname",
            "week",
            "final_points",
            "projected_points",
        ]
    ]
    one_playoff_season = one_playoff_season.rename(
        columns={
            "game_id_x": "game_id",
            "name": "team_name",
            "nickname": "team_nickname",
            "final_points": "team_pts",
            "projected_points": "team_pro_pts",
        }
    )
    one_playoff_season.sort_values(['team_key', 'week'], inplace=True)
    one_playoff_season["ttl_pts_for_run"] = one_playoff_season.groupby(["team_key"])[
        "team_pts"
    ].cumsum()
    one_playoff_season["ttl_pro_pts_for_run"] = one_playoff_season.groupby(
        ["team_key"]
    )["team_pro_pts"].cumsum()

    one_playoff_season = one_playoff_season.merge(
        reg_season_rank,
        how="left",
        on=["game_id", "team_key", "team_name", "team_nickname"],
    )

    one_playoff_season.rename(
        columns={"reg_season_rank_run": "reg_season_rank"}, inplace=True
    )

    one_playoff_season.sort_values(
        ["week", "reg_season_rank"], ascending=[True, True], inplace=True
    )

    DatabaseCursor(PATH, options=OPTION_PROD).copy_table_to_postgres_new(
        df=one_reg_season, table=f"reg_season_board_{str(key)}", first_time="YES"
    )
    DatabaseCursor(PATH, options=OPTION_PROD).copy_table_to_postgres_new(
        df=one_playoff_season, table=f"playoff_board_{str(key)}", first_time="YES"
    )

In [12]:
test = DatabaseCursor(PATH, options=OPTION_PROD).copy_data_from_postgres("SELECT * FROM prod.playoff_board_406")

Successfully pulled: SELECT * FROM prod.playoff_board_406


In [14]:
test

,game_id,team_key,team_name,team_nickname,week,team_pts,team_pro_pts,ttl_pts_for_run,ttl_pro_pts_for_run,reg_season_rank
0,406,406.l.103661.t.1,Fantasy Sidelines,Tim,16,114.98,103.87,114.98,103.87,1
1,406,406.l.103661.t.4,King of the Hill,Ryan,16,112.92,132.40,112.92,132.40,2
2,406,406.l.103661.t.7,Harris & Harris RBs at Law 👨🏾‍⚖️,Greg,16,125.00,123.71,125.00,123.71,3
3,406,406.l.103661.t.2,Dak to the Future,Wes,16,88.50,128.69,88.50,128.69,4
4,406,406.l.103661.t.6,Wide Out University,Chris,16,83.86,80.28,83.86,80.28,5
5,406,406.l.103661.t.3,Fightin’ Blue Cock/Chubb/Wood,Kevin,16,156.88,110.97,156.88,110.97,6
6,406,406.l.103661.t.8,Poor Decisions 😢,Pete,16,94.38,109.24,94.38,109.24,7
7,406,406.l.103661.t.9,Push Ups in Prison,Pat,16,124.54,117.68,124.54,117.68,8
8,406,406.l.103661.t.10,Team 10,James,16,100.70,103.60,100.70,103.60,9
9,406,406.l.103661.t.5,The Titanic,Jeremy,16,72.34,103.88,72.34,103.88,10


#MOM FFBL Playoff Bracket

MOM FFBL Draft Analysis/Trends Replacement

In [ ]:
"""
MoM FFBL Draft Analysis/Trends Replacement
"""
draft_query = """SELECT * FROM dev.draftresults"""
teams_query = """SELECT team_key
, game_id
, league_id
, "team_standings.outcome_totals.wins"
, "team_standings.outcome_totals.losses"
, number_of_moves
, number_of_trades
, nickname
, name
, draft_grade
, clinched_playoffs 
, "team_standings.playoff_seed"
, "team_standings.points_against" 
, "team_standings.points_for"
, "team_standings.rank"
FROM dev.leagueteams"""
players_query = """SELECT player_key
, "name.full"
, league_id
, game_id
, primary_position
, "bye_weeks.week"
, "draft_analysis.average_pick"
, "draft_analysis.average_round"
, "draft_analysis.percent_drafted"
 FROM dev.playerlist"""
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
draft_analysis = draft.merge(
    teams,
    how="left",
    left_on=["team_key", "game_id", "league_id"],
    right_on=["team_key", "game_id", "league_id"],
    suffixes=("_draft", "_teams"),
)
draft_analysis = draft_analysis.merge(
    players,
    how="left",
    left_on=["player_key", "game_id", "league_id"],
    right_on=["player_key", "game_id", "league_id"],
    suffixes=("", "_players"),
)
draft_analysis = draft_analysis[~draft_analysis["name"].isna()]
draft_analysis["clinched_playoffs"].fillna(0, inplace=True)
draft_analysis["team_standings.playoff_seed"].fillna(0, inplace=True)
encoded_grades = {
    "A+": 4,
    "A": 4,
    "A-": 3.7,
    "B+": 3.3,
    "B": 3.0,
    "B-": 2.7,
    "C+": 2.3,
    "C": 2.0,
    "C-": 1.7,
    "D+": 1.3,
    "D": 1.0,
    "na": np.nan,
}
draft_analysis["draft_gpa"] = draft_analysis["draft_grade"].replace(encoded_grades)
draft_analysis["avg_draft_gpa_rank"] = draft_analysis.groupby(["team_standings.rank"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_nickname"] = draft_analysis.groupby(["nickname"])[
    "draft_gpa"
].transform("mean")
draft_analysis["avg_draft_gpa_clinched_playoffs"] = draft_analysis.groupby(
    ["clinched_playoffs"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_playoffseed"] = draft_analysis.groupby(
    ["team_standings.playoff_seed"]
)["draft_gpa"].transform("mean")
draft_analysis["avg_draft_gpa_draft_pos"] = draft_analysis.groupby(["pick"])[
    "draft_gpa"
].transform("mean")

MoM FFBL League History Replacement

In [ ]:
"""
MoM FFBL League History Replacement
"""
matchups_query = """SELECT * FROM dev.weeklyleaguematchups"""
teams_query = """SELECT * FROM dev.leagueteams"""
draft_query = """SELECT * FROM dev.draftresults"""
players_query = """SELECT * FROM dev.playerlist"""
matchups = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    matchups_query
)
teams = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(teams_query)
draft = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(draft_query)
players = DatabaseCursor(PATH, options=OPTION_DEV).copy_data_from_postgres(
    players_query
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_a_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("", "_team_a"),
)
matchups = matchups.merge(
    teams,
    how="left",
    left_on=["game_id", "league_id", "team_b_team_key"],
    right_on=["game_id", "league_id", "team_key"],
    suffixes=("_team_a", "_team_b"),
)

matchups[
    (matchups["is_playoffs"] == 1) & (matchups["is_consolation"] == 0)
].sort_values(
    ["week_start", "is_consolation", "winner_team_key"], ascending=[False, True, True]
)

MoM FFBL League Records Replacement

In [ ]:
"""
MoM FFBL League Records Replacement
"""